<a href="https://colab.research.google.com/github/AteevMishra/Animal-Detection-Dog-or-Cat-/blob/master/Cats_and_Dogs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
print(tf.__version__)


2.2.0-rc2


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
import random
import time
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dropout,Dense,Activation,Flatten,Conv2D,MaxPooling2D
from tensorflow.keras.callbacks import TensorBoard

#NAME = 'Cats-vs-dog-cnn-64x2-{}'.format(int(time.time())) --This is for tensorboard************
#tensorboard = TensorBoard(log_dir = 'logs\{}'.format(NAME)) #callback
#gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction = 0.333)
#sess = tf.Session(config = tf.ConfigProto(gpu_options = gpu_options ))*************************

#DATADIR = "D:\Ateev\Kaggle\Cats and Dogs\dataset\PetImages"

_URL = 'https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip'
path_to_zip = tf.keras.utils.get_file('cats_and_dogs.zip', origin=_URL, extract=True)
PATH = os.path.join(os.path.dirname(path_to_zip), 'cats_and_dogs_filtered')
DATADIR = os.path.join(PATH,'train')
CATEGORIES = ["dogs","cats"]

training_data = []

def create_training_data():
    for category in CATEGORIES:
        path = os.path.join(DATADIR,category) #--path to cats or dogs dir
        class_num = CATEGORIES.index(category)
        for img in os.listdir(path):
            try:   #--We are using 'try' because some image might be broken
                img_array = cv2.imread(os.path.join(path,img), 0)#--instead of '0' you can also write -> cv2.IMREAD_GRAYSCALE)
                #plt.imshow(img_array,cmap = "gray")
                #plt.show()
                #print(img_array)
                #break
                IMG_SIZE = 100
                new_array = cv2.resize(img_array,(IMG_SIZE,IMG_SIZE))#--images are of different sizes so we need to make all of them of same size which is what we also call normalization of data
                training_data.append([new_array,class_num])  #--appending data in the training set
            except Exception as e:
                pass

create_training_data()
#print(len(training_data))

random.shuffle(training_data) #--Now we shuffle the data

IMG_SIZE = 100
X=[]
Y=[]
for features,label in training_data:
    X.append(features)
    Y.append(label)
X = np.array(X).reshape(-1,IMG_SIZE,IMG_SIZE,1) #--'1' because grayscale ,for RGB it would have been '3'

#--WE need to save our trainig data ,as we do not want to waste our time when we run the program again
np.save('features.npy',X) #--SAVING
#X=np.load('features.npy') <--LOADING
np.save('labels.npy',Y)

X=np.load('features.npy')
Y=np.load('labels.npy')

#before feeding the data into Neural Network we need to first 'normalise' the data
X=X/255.0 #--Normalising using 'Scaling'

model = Sequential()
model.add(Conv2D(64,(3,3), input_shape = X.shape[1:]))#--Adding a convolution layer of 64 nodes,3x3 window
model.add(Activation("relu")) #--Adding activation layer
model.add(MaxPooling2D(pool_size=(2,2))) #--pool size/window size = 2x2

model.add(Conv2D(64,(3,3)))#--Adding a convolution layer of 64 nodes,3x3 window
model.add(Activation("relu")) #--Adding activation layer
model.add(MaxPooling2D(pool_size=(2,2))) #--pool size/window size = 2x2

model.add(Flatten())#--'Convolution layer' is 2D while 'Dense layer' is not ,so we need to flatten the data.This converts our 3D feature maps to 1D feature vector

model.add(Dense(64)) #--Adding a dense layer of 64 nodes, for good measure
model.add(Activation('relu'))

model.add(Dense(1)) #--'Output Layer'
model.add(Activation('sigmoid'))

model.compile(loss="binary_crossentropy", #--we could have used 'sparse_categorical_...' but we used 'binary_...' since we have 2 classes(dogs and cats) only
              optimizer="adam",           #gradient descent optimization algo
              metrics=['accuracy'])

model.fit(X, Y, batch_size=32, epochs=7, validation_split=0.1)#--There are many samples;batch size defines how many samples to feed into the network at a time. 'validation_split' is out of sample data =0.3 means 30%

model.save('64x3-CNN.model')


Epoch 1/7
57/57 [==============================] - 1s 19ms/step - loss: 0.6903 - accuracy: 0.5233 - val_loss: 0.6849 - val_accuracy: 0.5800
Epoch 2/7
57/57 [==============================] - 1s 16ms/step - loss: 0.6556 - accuracy: 0.6300 - val_loss: 0.7107 - val_accuracy: 0.5650
Epoch 3/7
57/57 [==============================] - 1s 16ms/step - loss: 0.5804 - accuracy: 0.7033 - val_loss: 0.7319 - val_accuracy: 0.6000
Epoch 4/7
57/57 [==============================] - 1s 16ms/step - loss: 0.4790 - accuracy: 0.7783 - val_loss: 0.8187 - val_accuracy: 0.5900
Epoch 5/7
57/57 [==============================] - 1s 16ms/step - loss: 0.4102 - accuracy: 0.8256 - val_loss: 0.9980 - val_accuracy: 0.5600
Epoch 6/7
57/57 [==============================] - 1s 16ms/step - loss: 0.3187 - accuracy: 0.8689 - val_loss: 1.0002 - val_accuracy: 0.5700
Epoch 7/7
57/57 [==============================] - 1s 16ms/step - loss: 0.1924 - accuracy: 0.9328 - val_loss: 0.9850 - val_accuracy: 0.6750
INFO:tensorflow:Asse

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!ls '/content/drive/My Drive'

 cat.jpg	    download1.jpg  'Getting started.pdf'
'Colab Notebooks'   download3.jpg   images.jpg
 dog.jpg	    download.jpg    Untitled0.ipynb


In [ ]:
import tensorflow as tf
import cv2
filepath = "/content/drive/My Drive/dog.jpg"

def prepare(filepath):
  IMG_SIZE = 100
  img_array = cv2.imread(filepath,0)

  new_array = cv2.resize(img_array,(IMG_SIZE,IMG_SIZE))
  return new_array.reshape(-1,IMG_SIZE,IMG_SIZE,1)

model = tf.keras.models.load_model("64x3-CNN.model")
prediction = model.predict([prepare(filepath)])
animal = ['Cat','Dog']
print(CATEGORIES[int(prediction[0][0])])

cats
